In [1]:

from pyspark.sql import Window
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count, mean, stddev, min, max
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:

# Crie uma sessão Spark
spark = SparkSession.builder \
    .appName("Medicamentos ETL") \
    .getOrCreate()

# Carregue os dados
df = spark.read.csv("C:/Users/johnn/Documents/workspace/Projetos/dataframes/EDA_Manipulados_Combinado.csv", header=True, inferSchema=True)

# Mostre o esquema dos dados
df.printSchema()

root
 |-- ANO_VENDA: integer (nullable = true)
 |-- MES_VENDA: integer (nullable = true)
 |-- UF_VENDA: string (nullable = true)
 |-- MUNICIPIO_VENDA: string (nullable = true)
 |-- DCB: integer (nullable = true)
 |-- PRINCIPIO_ATIVO: string (nullable = true)
 |-- QTD_ATIVO_POR_UNID_FARMACOTEC: string (nullable = true)
 |-- UNIDADE_MEDIDA_PRINCIPIO_ATIVO: string (nullable = true)
 |-- QTD_UNIDADE_FARMACOTECNICA: string (nullable = true)
 |-- TIPO_UNIDADE_FARMACOTECNICA: string (nullable = true)
 |-- CONSELHO_PRESCRITOR: string (nullable = true)
 |-- UF_CONSELHO_PRESCRITOR: string (nullable = true)
 |-- TIPO_RECEITUARIO: integer (nullable = true)
 |-- CID10: string (nullable = true)
 |-- SEXO: double (nullable = true)
 |-- IDADE: double (nullable = true)
 |-- UNIDADE_IDADE: double (nullable = true)



In [3]:
# Número de linhas que você deseja visualizar
num_rows = 5

# Obter as últimas linhas
last_rows = df.tail(num_rows)
print(last_rows)


[Row(ANO_VENDA=2021, MES_VENDA=11, UF_VENDA='TO', MUNICIPIO_VENDA='PORTO NACIONAL', DCB=9113, PRINCIPIO_ATIVO='CLORIDRATO DE VENLAFAXINA', QTD_ATIVO_POR_UNID_FARMACOTEC='0,05', UNIDADE_MEDIDA_PRINCIPIO_ATIVO='GRAMA', QTD_UNIDADE_FARMACOTECNICA='1080', TIPO_UNIDADE_FARMACOTECNICA='CÁPSULA', CONSELHO_PRESCRITOR='CRM', UF_CONSELHO_PRESCRITOR='TO', TIPO_RECEITUARIO=1, CID10=None, SEXO=None, IDADE=None, UNIDADE_IDADE=None), Row(ANO_VENDA=2021, MES_VENDA=11, UF_VENDA='TO', MUNICIPIO_VENDA='PORTO NACIONAL', DCB=9296, PRINCIPIO_ATIVO='HEMITARTARATO DE ZOLPIDEM', QTD_ATIVO_POR_UNID_FARMACOTEC='0,005', UNIDADE_MEDIDA_PRINCIPIO_ATIVO='GRAMA', QTD_UNIDADE_FARMACOTECNICA='3510', TIPO_UNIDADE_FARMACOTECNICA='CÁPSULA', CONSELHO_PRESCRITOR='CRM', UF_CONSELHO_PRESCRITOR='TO', TIPO_RECEITUARIO=1, CID10=None, SEXO=None, IDADE=None, UNIDADE_IDADE=None), Row(ANO_VENDA=2021, MES_VENDA=11, UF_VENDA='TO', MUNICIPIO_VENDA='PORTO NACIONAL', DCB=9296, PRINCIPIO_ATIVO='HEMITARTARATO DE ZOLPIDEM', QTD_ATIVO_POR_UN

In [4]:
# Mostrar o esquema dos dados
df.printSchema()

# Mostrar as primeiras 5 linhas
df.show(5)







root
 |-- ANO_VENDA: integer (nullable = true)
 |-- MES_VENDA: integer (nullable = true)
 |-- UF_VENDA: string (nullable = true)
 |-- MUNICIPIO_VENDA: string (nullable = true)
 |-- DCB: integer (nullable = true)
 |-- PRINCIPIO_ATIVO: string (nullable = true)
 |-- QTD_ATIVO_POR_UNID_FARMACOTEC: string (nullable = true)
 |-- UNIDADE_MEDIDA_PRINCIPIO_ATIVO: string (nullable = true)
 |-- QTD_UNIDADE_FARMACOTECNICA: string (nullable = true)
 |-- TIPO_UNIDADE_FARMACOTECNICA: string (nullable = true)
 |-- CONSELHO_PRESCRITOR: string (nullable = true)
 |-- UF_CONSELHO_PRESCRITOR: string (nullable = true)
 |-- TIPO_RECEITUARIO: integer (nullable = true)
 |-- CID10: string (nullable = true)
 |-- SEXO: double (nullable = true)
 |-- IDADE: double (nullable = true)
 |-- UNIDADE_IDADE: double (nullable = true)

+---------+---------+--------+--------------------+----+--------------------+-----------------------------+------------------------------+--------------------------+--------------------------

In [5]:
total_linhas = df.count()
print(f"Total de linhas são: {total_linhas:,} linhas".replace(',', '.')) #o replace faz o número ficar separado por ponto 



Total de linhas são: 30.902.374 linhas


In [6]:

# Descrever as estatísticas do DataFrame
df.describe().show()


+-------+------------------+-----------------+--------+-------------------+-----------------+----------------+-----------------------------+------------------------------+--------------------------+---------------------------+-------------------+----------------------+------------------+-----+------------------+------------------+-------------------+
|summary|         ANO_VENDA|        MES_VENDA|UF_VENDA|    MUNICIPIO_VENDA|              DCB| PRINCIPIO_ATIVO|QTD_ATIVO_POR_UNID_FARMACOTEC|UNIDADE_MEDIDA_PRINCIPIO_ATIVO|QTD_UNIDADE_FARMACOTECNICA|TIPO_UNIDADE_FARMACOTECNICA|CONSELHO_PRESCRITOR|UF_CONSELHO_PRESCRITOR|  TIPO_RECEITUARIO|CID10|              SEXO|             IDADE|      UNIDADE_IDADE|
+-------+------------------+-----------------+--------+-------------------+-----------------+----------------+-----------------------------+------------------------------+--------------------------+---------------------------+-------------------+----------------------+------------------+-----+

In [7]:

# ver as colunas do DataFrame
df.columns

['ANO_VENDA',
 'MES_VENDA',
 'UF_VENDA',
 'MUNICIPIO_VENDA',
 'DCB',
 'PRINCIPIO_ATIVO',
 'QTD_ATIVO_POR_UNID_FARMACOTEC',
 'UNIDADE_MEDIDA_PRINCIPIO_ATIVO',
 'QTD_UNIDADE_FARMACOTECNICA',
 'TIPO_UNIDADE_FARMACOTECNICA',
 'CONSELHO_PRESCRITOR',
 'UF_CONSELHO_PRESCRITOR',
 'TIPO_RECEITUARIO',
 'CID10',
 'SEXO',
 'IDADE',
 'UNIDADE_IDADE']

In [8]:
from pyspark.sql.functions import col, isnan, when, count

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()



+---------+---------+--------+---------------+---+---------------+-----------------------------+------------------------------+--------------------------+---------------------------+-------------------+----------------------+----------------+--------+--------+--------+-------------+
|ANO_VENDA|MES_VENDA|UF_VENDA|MUNICIPIO_VENDA|DCB|PRINCIPIO_ATIVO|QTD_ATIVO_POR_UNID_FARMACOTEC|UNIDADE_MEDIDA_PRINCIPIO_ATIVO|QTD_UNIDADE_FARMACOTECNICA|TIPO_UNIDADE_FARMACOTECNICA|CONSELHO_PRESCRITOR|UF_CONSELHO_PRESCRITOR|TIPO_RECEITUARIO|   CID10|    SEXO|   IDADE|UNIDADE_IDADE|
+---------+---------+--------+---------------+---+---------------+-----------------------------+------------------------------+--------------------------+---------------------------+-------------------+----------------------+----------------+--------+--------+--------+-------------+
|        0|        0|       0|              0|  0|              0|                            0|                             0|                     

In [9]:
# Estatísticas para uma coluna específica
df.select(mean(col("IDADE")), stddev(col("IDADE")), min(col("IDADE")), max(col("IDADE"))).show()

+-----------------+------------------+----------+----------+
|       avg(IDADE)|     stddev(IDADE)|min(IDADE)|max(IDADE)|
+-----------------+------------------+----------+----------+
|29.95378318157391|27.529553555949565|       0.0|     999.0|
+-----------------+------------------+----------+----------+



In [11]:
# Contagem de registros por UF_VENDA
df.groupBy("UF_VENDA").count().orderBy("count", ascending=False).show()

# Contagem de registros por CONSELHO_PRESCRITOR
df.groupBy("CONSELHO_PRESCRITOR").count().orderBy("count", ascending=False).show()


+--------+--------+
|UF_VENDA|   count|
+--------+--------+
|      SP|11985100|
|      MG| 3376593|
|      PR| 3029792|
|      GO| 2169377|
|      RJ| 1832759|
|      RS| 1143495|
|      SC| 1027525|
|      ES|  909037|
|      BA|  897752|
|      MS|  660455|
|      PE|  556023|
|      MT|  473171|
|      DF|  413398|
|      RN|  366595|
|      PA|  328584|
|      PB|  265531|
|      RO|  255960|
|      AL|  192171|
|      CE|  189341|
|      PI|  165203|
+--------+--------+
only showing top 20 rows

+-------------------+--------+
|CONSELHO_PRESCRITOR|   count|
+-------------------+--------+
|                CRM|29459785|
|               CRMV|  998937|
|                CRO|  236470|
|                RMS|  207182|
+-------------------+--------+



In [14]:
from pyspark.sql.functions import col

# Identificar os 10 princípios ativos mais comercializados
principios_ativos_top10 = df.groupBy("PRINCIPIO_ATIVO").count().orderBy("count", ascending=False).limit(10)
principios_ativos_top10.show()


+--------------------+-------+
|     PRINCIPIO_ATIVO|  count|
+--------------------+-------+
|CLORIDRATO DE FLU...|1201541|
|CLORIDRATO DE AMI...|1156914|
|ESTOLATO DE ERITR...|1083061|
|        TESTOSTERONA|1039520|
|CLORIDRATO DE BUP...|1013388|
|          TOPIRAMATO| 980906|
|CLORIDRATO DE SER...| 941997|
|CLORIDRATO DE PAR...| 843300|
|         GABAPENTINA| 821585|
|CLORIDRATO DE TRA...| 787330|
+--------------------+-------+



In [15]:
# Configuração para exibir mais caracteres nas colunas
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 1000)  # Para exibir mais linhas
spark.conf.set("spark.sql.repl.eagerEval.truncate", 1000)    # Definir um valor alto para evitar truncamento

# Mostrar novamente os 10 princípios ativos mais comercializados
principios_ativos_top10.show(truncate=False)


+---------------------------+-------+
|PRINCIPIO_ATIVO            |count  |
+---------------------------+-------+
|CLORIDRATO DE FLUOXETINA   |1201541|
|CLORIDRATO DE AMITRIPTILINA|1156914|
|ESTOLATO DE ERITROMICINA   |1083061|
|TESTOSTERONA               |1039520|
|CLORIDRATO DE BUPROPIONA   |1013388|
|TOPIRAMATO                 |980906 |
|CLORIDRATO DE SERTRALINA   |941997 |
|CLORIDRATO DE PAROXETINA   |843300 |
|GABAPENTINA                |821585 |
|CLORIDRATO DE TRAMADOL     |787330 |
+---------------------------+-------+



In [23]:


# Calcular a contagem de prescrições por UF para cada princípio ativo
consumo_por_uf = df_top10.groupBy("PRINCIPIO_ATIVO", "UF_VENDA").count().orderBy("count", ascending=False)

# Mostrar os resultados
consumo_por_uf.show(50, truncate=False)  # como o pyspark trunca texto por padrão, o truncate=False mostra o nome dos principios ativos sem truncar


+---------------------------+--------+------+
|PRINCIPIO_ATIVO            |UF_VENDA|count |
+---------------------------+--------+------+
|CLORIDRATO DE FLUOXETINA   |SP      |476742|
|ESTOLATO DE ERITROMICINA   |SP      |472509|
|CLORIDRATO DE AMITRIPTILINA|SP      |448308|
|CLORIDRATO DE BUPROPIONA   |SP      |421473|
|TOPIRAMATO                 |SP      |416011|
|GABAPENTINA                |SP      |364965|
|TESTOSTERONA               |SP      |344765|
|CLORIDRATO DE PAROXETINA   |SP      |299681|
|CLORIDRATO DE TRAMADOL     |SP      |292596|
|CLORIDRATO DE SERTRALINA   |SP      |291023|
|CLORIDRATO DE SERTRALINA   |MG      |132145|
|CLORIDRATO DE AMITRIPTILINA|GO      |131630|
|CLORIDRATO DE SERTRALINA   |PR      |131183|
|CLORIDRATO DE FLUOXETINA   |GO      |130710|
|CLORIDRATO DE TRAMADOL     |GO      |129696|
|CLORIDRATO DE AMITRIPTILINA|MG      |124783|
|CLORIDRATO DE FLUOXETINA   |MG      |119490|
|CLORIDRATO DE PAROXETINA   |MG      |113311|
|CLORIDRATO DE BUPROPIONA   |GO   

In [24]:
# Obter os valores únicos na coluna PRINCIPIO_ATIVO
principios_ativos_unicos = df.select("PRINCIPIO_ATIVO").distinct().collect()

# Mostrar os valores únicos
for row in principios_ativos_unicos:
    print(row["PRINCIPIO_ATIVO"])


LEVOFOLINATO DE CÁLCIO
CLORIDRATO DE TETRACICLINA
OLEATO DE ETILA
LAURILSULFONATO DE MIRTECAÍNA
CLORIDRATO DE MINOCICLINA
NITROFURANTOÍNA
MESILATO DE ISOETARINA
CLORIDRATO DE NORTRIPTILINA
EPERVUDINA
DARODIPINO
SATRAPLATINA
DOXICICLINA MONOIDRATADA
CLORIDRATO DE NALOXONA
CLORIDRATO DE MEMANTINA
DICLORIDRATO DE TRIFLUOPERAZINA
ERGOTOXINA
CLOVOXAMINA
PERÓXIDO DE BENZOÍLA
BUSPIRONA
RISPENZEPINA
CITASAMICINA
ESTRADIOL
ORNOPROSTILA
SELEGILINA
GRISEOFULVINA
LEVOMEPROMAZINA
NIALAMIDA
DIETILESTILBESTROL
METRONIDAZOL
CLORIDRATO DE PSEUDOEFEDRINA
AMOXICILINA
ACISTRATO DE ERITROMICINA
HICLATO DE DOXICICLINA
CLOMIPRAMINA
NONABINA
AZIPRAMINA
HALOCORTOLONA
TARTARATO DE ERGOMETRINA
FLUCONAZOL
TRAZODONA
CLORIDRATO DE DESIPRAMINA
MIGLUSTATE
OXITETRACICLINA
HEMIFUMARATO DE QUETIAPINA
BROMIDRATO DE NORPIPANONA
ESTANOZOLOL
CLORIDRATO DE CLINDAMICINA
FOSFATO DISSÓDICO DE BETAMETASONA
ACETATO DE FLUAZACORTE
BIPERIDENO
PEPLOMICINA
EFEDRINA
ATAMESTANO
CLORIDRATO DE VENLAFAXINA
DESMETILMORAMIDA
CARABERSATE
SUL

In [27]:
from pyspark.sql.functions import col, count

# 1. Obter os 10 princípios ativos mais comercializados
top10_principios = df.groupBy("PRINCIPIO_ATIVO").count().orderBy("count", ascending=False).limit(10)

# 2. Filtrar o DataFrame para considerar apenas os top 10 princípios ativos
df_top10 = df.join(top10_principios, on="PRINCIPIO_ATIVO")

# 3. Calcular o número de prescrições por prescritor e por princípio ativo
prescricoes_por_prescritor = df_top10.groupBy("PRINCIPIO_ATIVO", "CONSELHO_PRESCRITOR") \
    .agg(count("*").alias("num_prescricoes"))

# 4. Ordenar os resultados por princípio ativo e, opcionalmente, por número de prescrições
prescricoes_por_prescritor_sorted = prescricoes_por_prescritor \
    .orderBy(col("PRINCIPIO_ATIVO"), col("num_prescricoes").desc())

# 5. Mostrar os resultados
prescricoes_por_prescritor_sorted.show(truncate=False)



+---------------------------+-------------------+---------------+
|PRINCIPIO_ATIVO            |CONSELHO_PRESCRITOR|num_prescricoes|
+---------------------------+-------------------+---------------+
|CLORIDRATO DE AMITRIPTILINA|CRM                |1073321        |
|CLORIDRATO DE AMITRIPTILINA|CRMV               |58463          |
|CLORIDRATO DE AMITRIPTILINA|RMS                |16561          |
|CLORIDRATO DE AMITRIPTILINA|CRO                |8569           |
|CLORIDRATO DE BUPROPIONA   |CRM                |999105         |
|CLORIDRATO DE BUPROPIONA   |CRO                |6973           |
|CLORIDRATO DE BUPROPIONA   |RMS                |4869           |
|CLORIDRATO DE BUPROPIONA   |CRMV               |2441           |
|CLORIDRATO DE FLUOXETINA   |CRM                |1115986        |
|CLORIDRATO DE FLUOXETINA   |CRMV               |61617          |
|CLORIDRATO DE FLUOXETINA   |CRO                |12347          |
|CLORIDRATO DE FLUOXETINA   |RMS                |11591          |
|CLORIDRAT

In [28]:
from pyspark.sql.functions import col, count, countDistinct

# 1. Obter os 10 princípios ativos mais comercializados
top10_principios = df.groupBy("PRINCIPIO_ATIVO").count().orderBy("count", ascending=False).limit(10)

# 2. Filtrar o DataFrame para considerar apenas os top 10 princípios ativos
df_top10 = df.join(top10_principios, on="PRINCIPIO_ATIVO")

# 3. Calcular o número de prescrições por prescritor e por princípio ativo
prescricoes_por_prescritor = df_top10.groupBy("PRINCIPIO_ATIVO", "CONSELHO_PRESCRITOR") \
    .agg(count("*").alias("num_prescricoes"))

# 4. Calcular o total de prescrições para cada princípio ativo
total_prescricoes_por_principio = prescricoes_por_prescritor.groupBy("PRINCIPIO_ATIVO") \
    .agg(count("num_prescricoes").alias("total_prescricoes"))

# 5. Juntar os dados de prescrições por prescritor com o total de prescrições
prescricoes_com_totais = prescricoes_por_prescritor.join(total_prescricoes_por_principio, on="PRINCIPIO_ATIVO")

# 6. Calcular a porcentagem de prescrições
prescricoes_com_porcentagem = prescricoes_com_totais \
    .withColumn("percentagem", (col("num_prescricoes") / col("total_prescricoes")) * 100)

# 7. Ordenar os resultados por princípio ativo e, opcionalmente, por número de prescrições
resultado_final = prescricoes_com_porcentagem \
    .orderBy(col("PRINCIPIO_ATIVO"), col("percentagem").desc())

# 8. Mostrar os resultados
resultado_final.show(truncate=False)


+---------------------------+-------------------+---------------+-----------------+-----------+
|PRINCIPIO_ATIVO            |CONSELHO_PRESCRITOR|num_prescricoes|total_prescricoes|percentagem|
+---------------------------+-------------------+---------------+-----------------+-----------+
|CLORIDRATO DE AMITRIPTILINA|CRM                |1073321        |4                |2.6833025E7|
|CLORIDRATO DE AMITRIPTILINA|CRMV               |58463          |4                |1461575.0  |
|CLORIDRATO DE AMITRIPTILINA|RMS                |16561          |4                |414025.0   |
|CLORIDRATO DE AMITRIPTILINA|CRO                |8569           |4                |214225.0   |
|CLORIDRATO DE BUPROPIONA   |CRM                |999105         |4                |2.4977625E7|
|CLORIDRATO DE BUPROPIONA   |CRO                |6973           |4                |174325.0   |
|CLORIDRATO DE BUPROPIONA   |RMS                |4869           |4                |121725.0   |
|CLORIDRATO DE BUPROPIONA   |CRMV       